# STA/LTA: Detección de eventos por medio de ventanas.

In [46]:
import os
import csv
from obspy import read
from obspy.signal.trigger import classic_sta_lta, trigger_onset

root_directory = "tests"
output_csv = "resultados_eventos_relativos.csv"


with open(output_csv, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["filename", "time_rel(sec)"])
    for subdir, dirs, files in os.walk(root_directory):
        for file_name in files:
            if file_name.endswith(".mseed"):
                file_path = os.path.join(subdir, file_name)
                
        
                st = read(file_path)
                tr = st[0]

                # Filtrar la señal
                tr_filt = tr.filter("bandpass", freqmin=1, freqmax=10)
                
                # STA/LTA
                sta = 3  # Ventana corta 
                lta = 50  # Ventana larga
                cft = classic_sta_lta(tr_filt.data, int(sta * tr_filt.stats.sampling_rate), int(lta * tr_filt.stats.sampling_rate))
                on_trig = 3.5  # Umbral-activación
                off_trig = 0.5  # Umbral-desactivación
                triggers = trigger_onset(cft, on_trig, off_trig)
                for event in triggers:
                    inicio_relativo = event[0] / tr_filt.stats.sampling_rate  
                    fin_relativo = event[1] / tr_filt.stats.sampling_rate   
                    t = (inicio_relativo + fin_relativo) / 2
                    writer.writerow([file_name, str(t)])
print(f"Procesamiento completo!!!! Los resultados se guardaron en {output_csv}")

/Users/rafa/anaconda3/lib/python3.11/site-packages/obspy/signal/filter.py:62: UserWarning: Selected high corner frequency (10) of bandpass is at or above Nyquist (3.3125). Applying a high-pass instead.
  warnings.warn(msg)


Procesamiento completo. Los resultados se guardaron en resultados_eventos_relativos.csv


# Autoencoder: Reducción de datos por medio de extracción de features con una Red Neuronal.

In [49]:
import numpy as np
from obspy import read
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.preprocessing import MinMaxScaler

st = read("XB.ELYSE.02.BHV.2022-02-03HR08_evid0005.mseed")
data = st[0].data  # Suponiendo que tienes una sola traza
tr = st[0]  
# Filtrar la señal (banda de paso entre 1-10 Hz)
tr_filt = tr.filter("bandpass", freqmin=1, freqmax=10)

# Definir la arquitectura del autoencoder
input_dim = tr_filt.data.shape[0]
encoding_dim = 32 

input_layer = Input(shape=(input_dim,))
encoded = Dense(encoding_dim, activation='relu')(input_layer)
decoded = Dense(input_dim, activation='sigmoid')(encoded)

# Crear el autoencoder
autoencoder = Model(input_layer, decoded)

# Compilar el autoencoder
autoencoder.compile(optimizer='adam', loss='mean_squared_error') # MSE para la reconstrucción

# Normalizar los datos
scaler = MinMaxScaler()
signal_data = tr_filt.data.reshape(-1, 1) # ---> para normalizar
signal_data_normalized = scaler.fit_transform(signal_data).reshape(1, -1)

# Entrenar el autoencoder con la señal del evento normalizada
autoencoder.fit(signal_data_normalized, signal_data_normalized, epochs=100, batch_size=128, shuffle=False)

# Usar el autoencoder para detectar si nuevas señales son similares al evento
reconstructed_signal = autoencoder.predict(signal_data_normalized)
reconstruction_error = np.mean(np.abs(signal_data_normalized - reconstructed_signal))

print(f"Error de reconstrucción: {reconstruction_error}")
# autoencoder.save("autoencoder_sismico.h5")


Epoch 1/100


/Users/rafa/anaconda3/lib/python3.11/site-packages/obspy/signal/filter.py:62: UserWarning: Selected high corner frequency (10) of bandpass is at or above Nyquist (10.0). Applying a high-pass instead.
  warnings.warn(msg)


1/1 [==============================] - 0s 201ms/step - loss: 0.0020
Epoch 2/100
1/1 [==============================] - 0s 8ms/step - loss: 0.0020
Epoch 3/100
1/1 [==============================] - 0s 11ms/step - loss: 0.0020
Epoch 4/100
1/1 [==============================] - 0s 8ms/step - loss: 0.0020
Epoch 5/100
1/1 [==============================] - 0s 8ms/step - loss: 0.0020
Epoch 6/100
1/1 [==============================] - 0s 8ms/step - loss: 0.0020
Epoch 7/100
1/1 [==============================] - 0s 7ms/step - loss: 0.0020
Epoch 8/100
1/1 [==============================] - 0s 9ms/step - loss: 0.0020
Epoch 9/100
1/1 [==============================] - 0s 7ms/step - loss: 0.0020
Epoch 10/100
1/1 [==============================] - 0s 9ms/step - loss: 0.0020
Epoch 11/100
1/1 [==============================] - 0s 7ms/step - loss: 0.0020
Epoch 12/100
1/1 [==============================] - 0s 9ms/step - loss: 0.0020
Epoch 13/100
1/1 [==============================] - 0s 7ms/step - loss

# Extra: Para mayor compresión se utiliza el Algoritmo de Hoffman.

In [50]:
import heapq
from collections import defaultdict

# Cuantización de la matriz
def quantize_signal(signal, num_levels=256):
    min_val = np.min(signal)
    max_val = np.max(signal)
    # Normalizar y escalar a un rango de [0, num_levels-1]
    quantized_signal = np.round((signal - min_val) / (max_val - min_val) * (num_levels - 1))
    return quantized_signal.astype(int)

# Aplanar la matriz de pesos
def flatten_weights(weights):
    return np.array(weights).flatten()

# Vamos a crear la clase del árbol de Huffman
class HuffmanNode:
    def __init__(self, symbol=None, freq=0, left=None, right=None):
        self.symbol = symbol
        self.freq = freq
        self.left = left
        self.right = right

    def __lt__(self, other):
        return self.freq < other.freq

# Árbol con las frecuencias
def build_huffman_tree(symbols_freq):
    heap = [HuffmanNode(symbol, freq) for symbol, freq in symbols_freq.items()]
    heapq.heapify(heap)

    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)
        merged = HuffmanNode(None, left.freq + right.freq, left, right)
        heapq.heappush(heap, merged)

    return heap[0]

# Generar el código Huffman
def generate_huffman_codes(node, prefix="", codebook={}):
    if node.symbol is not None:
        codebook[node.symbol] = prefix
    else:
        generate_huffman_codes(node.left, prefix + "0", codebook)
        generate_huffman_codes(node.right, prefix + "1", codebook)
    return codebook

# Compresión de la señal usando los códigos de Huffman
def huffman_encode(signal, codebook):
    encoded_signal = ''.join([codebook[val] for val in signal])
    return encoded_signal

# Decodificación del Huffman
def huffman_decode(encoded_signal, huffman_tree):
    decoded_signal = []
    current_node = huffman_tree
    for bit in encoded_signal:
        if bit == '0':
            current_node = current_node.left
        else:
            current_node = current_node.right

        if current_node.left is None and current_node.right is None:
            decoded_signal.append(current_node.symbol)
            current_node = huffman_tree  # Volvemos a la raíz

    return np.array(decoded_signal)

# Aplicación del algoritmo en la matriz de pesos del autoencoder
# Aquí asumimos que tienes una lista de capas con pesos y sesgos
for i, layer in enumerate(autoencoder.layers):
    weights = layer.get_weights()
    if len(weights) > 0:
        print(f"\nProcesando la capa {i} ({layer.name}):")
        # Cuantificar y aplanar la matriz de pesos
        quantized_weights = quantize_signal(flatten_weights(weights[0]))  # Solo procesamos los pesos
        print(f"Matriz de pesos cuantificada y aplanada:\n{quantized_weights}")

        # Contar las frecuencias de los símbolos en la señal cuantizada
        symbol_freq = defaultdict(int)
        for value in quantized_weights:
            symbol_freq[value] += 1

        # Construir el árbol de Huffman
        huffman_tree = build_huffman_tree(symbol_freq)

        # Generar los códigos de Huffman
        huffman_codes = generate_huffman_codes(huffman_tree)

        # Comprimir la señal cuantizada
        compressed_signal = huffman_encode(quantized_weights, huffman_codes)

        # Tamaños de la señal original y comprimida
        original_size = len(quantized_weights) * 8  # 8 bits por valor original
        compressed_size = len(compressed_signal)
        
        print(f"Tamaño de la señal original (bits): {original_size}")
        print(f"Tamaño de la señal comprimida (bits): {compressed_size}")
        print(f"Porcentaje de compresión: {(compressed_size / original_size) * 100:.4f}%")

        # Decodificar la señal comprimida
        decoded_signal = huffman_decode(compressed_signal, huffman_tree)
        is_identical = np.array_equal(decoded_signal, quantized_weights)
        print(f"Son iguales la señal decodificada y la original cuantificada: {'Sí' if is_identical else 'No'}")



Procesando la capa 1 (dense_4):
Matriz de pesos cuantificada y aplanada:
[144 194  63 ... 162  53 158]
Tamaño de la señal original (bits): 18432000
Tamaño de la señal comprimida (bits): 18220101
Porcentaje de compresión: 98.8504%
Son iguales la señal decodificada y la original cuantificada: Sí

Procesando la capa 2 (dense_5):
Matriz de pesos cuantificada y aplanada:
[170 105  45 ... 187 203  46]
Tamaño de la señal original (bits): 18432000
Tamaño de la señal comprimida (bits): 18020034
Porcentaje de compresión: 97.7649%
Son iguales la señal decodificada y la original cuantificada: Sí
